<a href="https://colab.research.google.com/github/sakshammverma/TurboAi/blob/main/TurboAI_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [240]:
import pandas as pd

df = pd.read_csv("/content/Comp_FoodDelivery_competition.csv")


In [241]:
df

,order_id,customer_city,restaurant_type,cuisine,order_value,delivery_time_min,payment_method,discount_percent,distance_km,is_peak_hours,customer_tenure_months,loyalty_tier,delivery_partner_rating,promo_code_used,device_type,high_value_ops,random_noise1,random_noise2
0,1001,Kolkata,Fine Dining,Thai,571,79,Card,35,6.38,1,30.0,Platinum,4.2,0,Android,1.0,1.800782,-0.334525
1,1002,Bengaluru,QSR,Chinese,458,22,Card,19,13.36,0,31.0,Bronze,4.4,1,iOS,1.0,1.023942,0.148138
2,1003,Chennai,Cloud Kitchen,Chinese,560,25,COD,41,13.30,0,41.0,Platinum,4.1,0,iOS,1.0,-1.084885,-0.031664
3,1004,Bengaluru,QSR,Chinese,583,61,COD,18,6.66,0,NaN,Gold,4.7,0,Web,1.0,-1.352203,-0.513860
4,1005,Mumbai,Casual Dining,Arabian,749,47,Wallet,43,11.87,1,0.0,Bronze,4.9,1,iOS,1.0,-0.161975,0.196649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1696,Kolkata,Cafe,Arabian,358,22,Wallet,48,6.61,0,45.0,Silver,4.2,1,Web,1.0,0.000000,0.323028
696,1697,Pune,Cloud Kitchen,Arabian,1383,21,Card,49,2.09,1,46.0,Platinum,3.5,1,Android,1.0,-0.269409,0.935373
697,1698,Delhi,Fine Dining,Mexican,405,22,Card,5,3.86,1,47.0,Gold,0.0,0,Web,1.0,0.336990,-0.065500
698,1699,Mumbai,Cafe,Indian,772,60,Card,20,2.64,1,12.0,Gold,3.7,1,iOS,1.0,0.057665,-0.668507


In [242]:
df.isnull().sum()

,0
order_id,0
customer_city,21
restaurant_type,0
cuisine,0
order_value,0
delivery_time_min,0
payment_method,0
discount_percent,0
distance_km,21
is_peak_hours,0


In [243]:
df["value_to_distance_ratio"] = df["order_value"] / (df["distance_km"] + 1)


In [244]:
df["speed"] = df["distance_km"] / (df["delivery_time_min"] + 1)


In [245]:
df = df.drop(['random_noise1', 'random_noise2', 'order_id', 'delivery_partner_rating'], axis=1)


In [246]:
df.head()

,customer_city,restaurant_type,cuisine,order_value,delivery_time_min,payment_method,discount_percent,distance_km,is_peak_hours,customer_tenure_months,loyalty_tier,promo_code_used,device_type,high_value_ops,value_to_distance_ratio,speed
0,Kolkata,Fine Dining,Thai,571,79,Card,35,6.38,1,30.0,Platinum,0,Android,1.0,77.371274,0.079750
1,Bengaluru,QSR,Chinese,458,22,Card,19,13.36,0,31.0,Bronze,1,iOS,1.0,31.894150,0.580870
2,Chennai,Cloud Kitchen,Chinese,560,25,COD,41,13.30,0,41.0,Platinum,0,iOS,1.0,39.160839,0.511538
3,Bengaluru,QSR,Chinese,583,61,COD,18,6.66,0,NaN,Gold,0,Web,1.0,76.109661,0.107419
4,Mumbai,Casual Dining,Arabian,749,47,Wallet,43,11.87,1,0.0,Bronze,1,iOS,1.0,58.197358,0.247292


In [247]:

df = df.drop_duplicates()
df = df.fillna(df.mode().iloc[0])


In [248]:
cat_cols = df.select_dtypes(include=["object"]).columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)


In [249]:
from sklearn.model_selection import train_test_split

X = df.drop("high_value_ops", axis=1)
y = df["high_value_ops"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=0)


In [250]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [251]:
from sklearn.metrics import accuracy_score

pred = model.predict(X_test)
print("Accuracy - ",accuracy_score(y_test, pred)*100)


Accuracy -  90.0


In [252]:

from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric="logloss"
)

model.fit(X_train, y_train)
pred = model.predict(X_test)
print(accuracy_score(y_test, pred)*100)


96.0


In [253]:
from sklearn.metrics import precision_score, f1_score, classification_report

print("Precision:", precision_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred))
print(classification_report(y_test, pred))


Precision: 0.9565217391304348
F1 Score: 0.9777777777777777
              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         6
         1.0       0.96      1.00      0.98        44

    accuracy                           0.96        50
   macro avg       0.98      0.83      0.89        50
weighted avg       0.96      0.96      0.96        50

